## Tensor Processing Units (TPUs) Usage in Google Colab

Reference: [Use TPUs - TensorFlow](https://www.tensorflow.org/guide/tpu#:~:text=and%20Cloud%20TPU.-,Setup,type%20%3E%20Hardware%20accelerator%20%3E%20TPU.)

In [1]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

### TPU initialization

TPUs are typically Cloud TPU workers, which are different from the local process running the user's Python program. Thus, you need to do some initialization work to connect to the remote cluster and initialize the TPUs.

`tf.distribute.cluster_resolver.TPUClusterResolver` is a special address just for Colab.

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


After the TPU is initialized, you can use manual device placement to place the computation on a single TPU device:

In [3]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

with tf.device('/TPU:0'):
  c = tf.matmul(a, b)

print("c device: ", c.device)
print(c)

c device:  /job:worker/replica:0/task:0/device:TPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


### Distribution strategies

Usually, you run your model on multiple TPUs in a `data-parallel` way. To distribute your model on multiple TPUs (as well as multiple GPUs or multiple machines), TensorFlow offers the `tf.distribute.Strategy` API. You can replace your distribution strategy and the model will run on any given (TPU) device.

Using the `tf.distribute.TPUStrategy` option implements `synchronous distributed training`. TPUs provide their own implementation of efficient all-reduce and other collective operations across multiple TPU cores, which are used in TPUStrategy.

To demonstrate this, create a tf.distribute.TPUStrategy object:

In [4]:
strategy = tf.distribute.TPUStrategy(resolver)

To replicate a computation so it can run in all TPU cores, you can pass it into the `Strategy.run` API. Below is an example that shows all cores `receiving` the `same inputs (a, b)` and performing matrix multiplication on each core independently. The outputs will be the values from all the replicas.

In [5]:
@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

z = strategy.run(matmul_fn, args=(a, b))
print(z)

PerReplica:{
  0: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  1: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  2: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  3: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  4: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  5: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  6: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  7: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
}


## Classification on TPUs

### Define a Model

Dataset: MNIST

Model: Convolutional Neural Network

In [6]:
def get_cnn_model():
  model= tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)
    ])

  model.summary()
  return model

### Load Dataset

Efficient use of the `tf.data.Dataset` API is critical when using a Cloud TPU. If you are using TPU Nodes, you need to `store all data files` read by the TensorFlow Dataset in `Google Cloud Storage (GCS) buckets`. If you are using TPU VMs, you can store data wherever you like.

For most use cases, it is recommended to `convert` your data into the `TFRecord` format and use a `tf.data.TFRecordDataset` to read it. You can `load entire small datasets` into memory using `tf.data.Dataset.cache`.

As shown in the code below, you should use the Tensorflow Datasets tfds.load module to get a copy of the MNIST training and test data. Note that `try_gcs` is specified to use a copy that is available in a public `GCS bucket`. If you don't specify this, the TPU will not be able to access the downloaded data.

In [7]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split, with_info=True,
                            as_supervised=True, try_gcs=True)

  # Normalize the input data.
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  dataset = dataset.map(scale)

  # Only shuffle and repeat the dataset in training. The advantage of having an
  # infinite dataset for training is to avoid the potential last partial batch
  # in each epoch, so that you don't need to think about scaling the gradients
  # based on the actual batch size.
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

### Train the Model

In [18]:
batch_size = 200
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

In [ ]:
with strategy.scope():
  model = get_cnn_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

In [ ]:
train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

In [8]:
model.fit(train_dataset, epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 256)       2560      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 256)       590080    
                                                                 
 flatten (Flatten)           (None, 147456)            0         
                                                                 
 dense (Dense)               (None, 256)               37748992  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 38,375,818
Trainable params: 38,375,818
No

To `reduce` Python overhead and `maximize` the `performance` of your TPU, pass in the `steps_per_execution` argument to Keras Model.compile. In this example, it increases throughput by about `50%`.

**steps_per_execution**: Int. The `number of batches` to run `during each tf.function call`. Running multiple batches inside a single tf.function call can greatly `improve performance` on TPUs or small models with a large Python overhead. At most, one full epoch will be run each execution. If a number larger than the size of the epoch is passed, the execution will be truncated to the size of the epoch. Note that if steps_per_execution is set to N, `Callback.on_batch_begin` and `Callback.on_batch_end` methods will `only be called` every `N batches` (i.e. before/after each tf.function execution). Defaults to **1**.

In [ ]:
with strategy.scope():
  model = get_cnn_model()
  model.compile(optimizer='adam',
                # Anything between 2 and `steps_per_epoch` could help here.
                steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

In [9]:
model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 256)       2560      
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 256)       590080    
                                                                 
 flatten_1 (Flatten)         (None, 147456)            0         
                                                                 
 dense_3 (Dense)             (None, 256)               37748992  
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 38,375,818
Trainable params: 38,375,818


### Training the model using a Custom Training loop

You can also create and train your model using `tf.function` and `tf.distribute` APIs directly. You can use the `Strategy.experimental_distribute_datasets_from_function` API to distribute the `tf.data.Dataset` given a dataset function. Note that in the example below the batch size passed into the Dataset is the per-replica batch size instead of the global batch size.

In [12]:
print(f'Number of parallel TPUs: {strategy.num_replicas_in_sync}, Batch size: {batch_size}')

Number of parallel TPUs: 8, Batch size: 200


In [13]:
# Process batches as much as per_replica_batch_size per TPU
per_replica_batch_size = batch_size // strategy.num_replicas_in_sync

In [15]:
with strategy.scope():
    model = get_cnn_model()
    optimizer = tf.keras.optimizers.Adam()
    training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
    training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      'training_accuracy', dtype=tf.float32)

    # Calculate per replica batch size, and distribute the `tf.data.Dataset`s
    # on each TPU worker.
    per_replica_batch_size = batch_size // strategy.num_replicas_in_sync

    train_dataset = strategy.distribute_datasets_from_function(
        lambda _: get_dataset(per_replica_batch_size, is_training=True))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 256)       2560      
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 256)       590080    
                                                                 
 flatten_3 (Flatten)         (None, 147456)            0         
                                                                 
 dense_9 (Dense)             (None, 256)               37748992  
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 10)                1290      
                                                                 
Total params: 38,375,818
Trainable params: 38,375,818


In [29]:
@tf.function
def train_step(iterator):
    """The step function for one training step."""
    def step_fn(inputs):
        """The computation to run on each TPU device."""
        images, labels = inputs
        with tf.GradientTape() as tape:
            logits = model(images, training=True)
            loss = tf.keras.losses.sparse_categorical_crossentropy(
                labels, logits, from_logits=True
            )
            loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))

            training_loss.update_state(loss*strategy.num_replicas_in_sync)
            training_accuracy.update_state(labels, logits)

    strategy.run(step_fn, args=(next(iterator),))

In [30]:
steps_per_eval = 10000 // batch_size

In [32]:
train_iterator = iter(train_dataset)

for epoch in range(5):
    print('Epoch: {}/5'.format(epoch))

    for step in range(steps_per_epoch):  # 60000 // batch_size
        train_step(train_iterator)

    print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))

    training_loss.reset_states()
    training_accuracy.reset_states()

Epoch: 0/5
Current step: 1829, training loss: 0.0085, accuracy: 99.69%
Epoch: 1/5
Current step: 2129, training loss: 0.0078, accuracy: 99.75%
Epoch: 2/5
Current step: 2429, training loss: 0.0059, accuracy: 99.8%
Epoch: 3/5
Current step: 2729, training loss: 0.0052, accuracy: 99.81%
Epoch: 4/5
Current step: 3029, training loss: 0.005, accuracy: 99.84%


### Improving performance with multiple steps inside tf.function

By running multiple steps within a tf.function, the performance can be improved. This is achieved by wrapping the `Strategy.run` call with a `tf.range` inside `tf.function`, and AutoGraph will convert it to a `tf.while_loop` on the TPU worker.

Note: Despite the improved performance, there are tradeoffs with this method compared to running a single step inside a tf.function. Running multiple steps in a tf.function is less flexible—you cannot run things eagerly or arbitrary Python code within the steps.

In [33]:
@tf.function
def train_multiple_steps(iterator, steps):
    """The step function for one training step."""
    def step_fn(inputs):
        """The computation to run on each TPU device."""
        images, labels = inputs
        with tf.GradientTape() as tape:
          logits = model(images, training=True)
          loss = tf.keras.losses.sparse_categorical_crossentropy(
              labels, logits, from_logits=True)
          loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))

        training_loss.update_state(loss * strategy.num_replicas_in_sync)
        training_accuracy.update_state(labels, logits)

    for _ in tf.range(steps):
      strategy.run(step_fn, args=(next(iterator),))

In [34]:
# Convert `steps_per_epoch` to `tf.Tensor` so the `tf.function` won't get
# retraced if the value changes.
train_multiple_steps(train_iterator, tf.convert_to_tensor(steps_per_epoch))

print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))

Current step: 3329, training loss: 0.0052, accuracy: 99.84%


### Reference

https://www.tensorflow.org/guide/tpu